### The Battle of Neighborhoods - New York City and Toronto (Week1) 


In [1]:
import pandas as pd
import numpy as np
import json, requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('  <<<<<<<<<<     Downloaded!     >>>>>>>>>>  ')

  <<<<<<<<<<     Downloaded!     >>>>>>>>>>  


In [4]:
with open('newyork_data.json') as json_data:
    nyc_json = json.load(json_data)
nyc_json = nyc_json['features']

cols = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
nyc_df = pd.DataFrame(columns=cols)

for data in nyc_json:
    borough = data['properties']['borough'] 
    name = data['properties']['name']
        
    latlon = data['geometry']['coordinates']
    lat = latlon[1]
    lon = latlon[0]
    
    nyc_df = nyc_df.append({'Borough': borough,
                          'Neighborhood': name,
                          'Latitude': lat,
                          'Longitude': lon}, ignore_index=True)
nyc_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Toronto, Canada - City Data

In [5]:
coordinates_df = pd.read_csv('coordinates.csv')
coordinates_df.head(15)

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,1,1,M5L,Downtown Toronto,Commerce Court,43.648198,-79.379817
2,2,2,M2R,North York,Willowdale West,43.782736,-79.442259
3,3,3,M8V,Etobicoke,New Toronto,43.605647,-79.501321
4,4,4,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849
5,5,5,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
6,6,6,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307
7,7,7,M4H,East York,Thorncliffe Park,43.705369,-79.349372
8,8,8,M3A,North York,Parkwoods,43.753259,-79.329656
9,9,9,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763


In [6]:
coordinates_df.shape

(77, 7)

In [8]:
CLIENT_ID = 'HNSCZV3FCJQ3D1W4AZHNXBMKQHCH0DKNMVJITR2FMMFA2ACJ' # your Foursquare ID
CLIENT_SECRET = 'HSHSVJ2RQ1Q01CIBPHCJNTMUHW1ZGDIMTVEBL3F01LLKVMKO' # your Foursquare Secret
VERSION = '20200221' # Foursquare API version
LIMIT = 100
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')


>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Your credentails:
CLIENT_ID: HNSCZV3FCJQ3D1W4AZHNXBMKQHCH0DKNMVJITR2FMMFA2ACJ
CLIENT_SECRET:HSHSVJ2RQ1Q01CIBPHCJNTMUHW1ZGDIMTVEBL3F01LLKVMKO
>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [10]:
def get_near_by_venues(names, latitudes, longitudes, radius= 1000):    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)    

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
                             v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue in venues_list for item in venue])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return nearby_venues

In [11]:
def explore_borough(df, borough):
    new_df = df[df['Borough'] == borough].reset_index(drop = True)
    venues =  get_near_by_venues(names = new_df['Neighborhood'],latitudes = new_df['Latitude'],
                                 longitudes = new_df['Longitude'])

    onehot_df = pd.get_dummies(venues[['Venue Category']], prefix= "", prefix_sep= "")

    # # add neighborhood column back to dataframe
    onehot_df['Neighborhood'] = venues['Neighborhood']
    # move neighborhood column to the first column
    fixed_columns = [onehot_df.columns[-1]] + list(onehot_df.columns[:-1])
    onehot_df = onehot_df[fixed_columns]
    onehot_df_grouped = onehot_df.groupby('Neighborhood').mean().reset_index()
    
    onehot_coded_df = pd.merge(new_df, onehot_df_grouped, on = 'Neighborhood', how = 'left')
    
    return onehot_coded_df

In [12]:
def return_venues_for_boroughs(df):
    col = list(df.columns)
    all_venues = pd.DataFrame()
    for i in df['Borough'].unique():
        new_df = df[df['Borough'] == i]
        print('For borough: ',i)
        a = explore_borough(new_df, i)
        all_venues = pd.concat([a, all_venues], axis = 0, ignore_index = True, sort = True)
    cols = col + [j for j in all_venues.columns if j not in col]
    all_venues = all_venues[cols]
    return all_venues

### retrieve NYC venue information

In [14]:

try:
    df1 = pd.read_csv('New_York_venue.csv')
except:
    df1 = return_venues_for_boroughs(nyc_df)
    df1.fillna(0,  inplace = True)
    df1.to_csv('New_York_venue.csv',  index_label = False)
print('Shape data is: ', df1.shape)
df1.head()

For borough:  Bronx
For borough:  Manhattan
For borough:  Brooklyn
For borough:  Queens
For borough:  Staten Island
Shape data is:  (306, 479)


,Borough,Neighborhood,Latitude,Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Gym,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Eur

In [38]:
try:
    df2 = pd.read_csv('coordinates.csv')
except:
    df2 = return_venues_for_boroughs(df_toronto)
    df2.fillna(0, inplace = True)
    df2.to_csv('coordinates.csv',  index_label = False)
print('Shape data is: ', df2.shape)
df2.head(15)

Shape data is:  (77, 7)


,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,1,1,M5L,Downtown Toronto,Commerce Court,43.648198,-79.379817
2,2,2,M2R,North York,Willowdale West,43.782736,-79.442259
3,3,3,M8V,Etobicoke,New Toronto,43.605647,-79.501321
4,4,4,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849
5,5,5,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
6,6,6,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307
7,7,7,M4H,East York,Thorncliffe Park,43.705369,-79.349372
8,8,8,M3A,North York,Parkwoods,43.753259,-79.329656
9,9,9,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
